In this notebook, we will be using sql to query data and create an overview table of candidate expenditures over a period of 12  months.
Data: 2016 Presidential Candidate Spending, records are as of mid May 2016. Data comes from the Federal Election Commission. [source](https://public.tableau.com/en-us/s/resources)

In [1]:
import pandas as pd
import pandasql as psql
from bs4 import BeautifulSoup
from googlesearch import search

In [2]:
# import pandassql and declare global
from pandasql import sqldf 
mysql = lambda q: sqldf(q, globals())

In [3]:
exp = pd.read_csv('2016_presidential_candidate_expenditures.csv')

In [4]:
exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159926 entries, 0 to 159925
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   cmte_id         159926 non-null  object 
 1   cand_id         159926 non-null  object 
 2   cand_nm         159926 non-null  object 
 3   Party           159926 non-null  object 
 4   recipient_nm    159926 non-null  object 
 5   disb_amt        159926 non-null  float64
 6   disb_dt         159926 non-null  object 
 7   recipient_city  159905 non-null  object 
 8   recipient_st    159926 non-null  object 
 9   recipient_zip   159862 non-null  object 
 10  disb_desc       159891 non-null  object 
 11  memo_cd         96680 non-null   object 
 12  memo_text       17812 non-null   object 
 13  form_tp         159926 non-null  object 
 14  file_num        159926 non-null  int64  
 15  tran_id         159926 non-null  object 
 16  election_tp     119169 non-null  object 
dtypes: float64

In [6]:
# select the first five rows from the dataframe. Looks like Party column misses values
mysql("SELECT * FROM exp LIMIT 5;")

,cmte_id,cand_id,cand_nm,Party,recipient_nm,disb_amt,disb_dt,recipient_city,recipient_st,recipient_zip,disb_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
0,C00458844,P60006723,"Rubio, Marco",#REF!,"GREEN, PERRY MR.",500.00,15-Sep-15,ANCHORAGE,AK,995084442,IN-KIND CONTRIBUTION,None,IN KIND: FACILITY RENTAL,SB23,1031526,SB23.831217,P2016
1,C00458844,P60006723,"Rubio, Marco",#REF!,ALASKA REPUBLICAN PARTY,2500.00,16-Nov-15,ANCHORAGE,AK,99503,BALLOT ACCESS,None,None,SB23,1047126,SB23.I14767,None
2,C00458844,P60006723,"Rubio, Marco",#REF!,"JOHNSON, MARK",1000.00,30-Sep-15,ANCHORAGE,AK,995163436,IN-KIND CONTRIBUTION,None,IN KIND: BUMPER STICKERS SIGNS AND BUTTONS,SB23,1031526,SB23.831218,P2016
3,C00458844,P60006723,"Rubio, Marco",#REF!,ENTERPRISE,336.09,18-Dec-15,BIRMINGHAM,AL,352124572,CAR RENTAL,X,None,SB23,1047126,SB23.I15112,None
4,C00458844,P60006723,"Rubio, Marco",#REF!,ALABAMA REPUBLICAN PARTY,10000.00,22-Oct-15,BIRMINGHAM,AL,35216,BALLOT ACCESS,None,None,SB23,1047126,SB23.I14766,None


In [98]:
# we'd like to know the candidate expenditure amount over the time
# create a dataframe that contains columns: cand_nm, disb_amt, disb_dt, recipient_st
exp_dtls = mysql('''SELECT cand_nm
                            , disb_amt
                            , disb_dt
                            , recipient_st
                            , party
                        FROM exp;''')

In [93]:
mysql('''SELECT cand_nm
        , Party
        , disb_amt
        , disb_dt
        , recipient_st
FROM exp LIMIT 5;''')

,cand_nm,Party,disb_amt,disb_dt,recipient_st
0,"Rubio, Marco",#REF!,500.00,15-Sep-15,AK
1,"Rubio, Marco",#REF!,2500.00,16-Nov-15,AK
2,"Rubio, Marco",#REF!,1000.00,30-Sep-15,AK
3,"Rubio, Marco",#REF!,336.09,18-Dec-15,AL
4,"Rubio, Marco",#REF!,10000.00,22-Oct-15,AL


In [70]:
exp_dtls.head()

,cand_nm,disb_amt,disb_dt,recipient_st
0,"Rubio, Marco",500.00,15-Sep-15,AK
1,"Rubio, Marco",2500.00,16-Nov-15,AK
2,"Rubio, Marco",1000.00,30-Sep-15,AK
3,"Rubio, Marco",336.09,18-Dec-15,AL
4,"Rubio, Marco",10000.00,22-Oct-15,AL


In [71]:
cand_ls = list(exp_dtls['cand_nm'].unique())
print(cand_ls)

['Rubio, Marco', 'Santorum, Richard J.', 'Perry, James R. (Rick)', 'Carson, Benjamin S.', "Cruz, Rafael Edward 'Ted'", 'Paul, Rand', 'Clinton, Hillary Rodham', 'Sanders, Bernard', 'Fiorina, Carly', 'Huckabee, Mike', 'Pataki, George E.', "O'Malley, Martin Joseph", 'Graham, Lindsey O.', 'Bush, Jeb', 'Trump, Donald J.', 'Jindal, Bobby', 'Christie, Christopher J.', 'Walker, Scott', 'Stein, Jill', 'Webb, James Henry Jr.', 'Kasich, John R.', 'Gilmore, James S IIII', 'Lessig, Lawrence', 'Johnson, Gary']


google search query and params:<br>
query -- <br>
search(query, tld='com', lang='en', num=10, start=0, stop=None, pause=2.0)

params-- <br>
query : query string that we want to search for.
tld : top level domain search in which google domain.
lang : language.
num : Number of results .
start : First result to retrieve.
stop : Last result to retrieve. 
    Use None to keep searching forever.
pause : Lapse to wait between HTTP requests. Lapse too short may cause Google to block your IP. 
    Keeping significant lapse will make your program slow but its safe and better option.
Return : Generator (iterator) that yields found URLs. 
    If the stop parameter is None the iterator will loop forever.


In [10]:
#  search candidates' names in google and create a list of parties
parties = []
for cand in cand_ls:
    for i in list(search(cand+" Party", tld='com', lang='en', num=1, stop=1, pause=2.0)):
        parties.append(i)

In [11]:
party_ls = []
for party in parties:
    if "Republicanlogo" in party:
        party_ls.append("Republican")
    elif "DemocraticLogo" in party:
        party_ls.append("Democratic")
    elif "Libertarian_Party" in party:
        party_ls.append("Libertarian")
    elif "Bernie_Sanders" in party:
        party_ls.append("Democratic")
    elif "Jill_Stein" in party:
        party_ls.append("Green-Rainbow")
    elif "Jim_Gilmore" in party:
        party_ls.append("Republican")
    else:
        party_ls.append(party)
print(party_ls)

['Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Democratic', 'Democratic', 'Republican', 'Republican', 'Republican', 'Democratic', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Green-Rainbow', 'Democratic', 'Republican', 'Republican', 'Democratic', 'Libertarian']


In [12]:
# create a dataframe consists of candidates name and parties
cand_pt_dctnry = {'candidate_nm':cand_ls,'party_nm':party_ls}
print(cand_pt_dctnry)

{'candidate_nm': ['Rubio, Marco', 'Santorum, Richard J.', 'Perry, James R. (Rick)', 'Carson, Benjamin S.', "Cruz, Rafael Edward 'Ted'", 'Paul, Rand', 'Clinton, Hillary Rodham', 'Sanders, Bernard', 'Fiorina, Carly', 'Huckabee, Mike', 'Pataki, George E.', "O'Malley, Martin Joseph", 'Graham, Lindsey O.', 'Bush, Jeb', 'Trump, Donald J.', 'Jindal, Bobby', 'Christie, Christopher J.', 'Walker, Scott', 'Stein, Jill', 'Webb, James Henry Jr.', 'Kasich, John R.', 'Gilmore, James S IIII', 'Lessig, Lawrence', 'Johnson, Gary'], 'party_nm': ['Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Democratic', 'Democratic', 'Republican', 'Republican', 'Republican', 'Democratic', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Green-Rainbow', 'Democratic', 'Republican', 'Republican', 'Democratic', 'Libertarian']}


In [13]:
cand_pt = pd.DataFrame.from_dict(cand_pt_dctnry)
mysql("SELECT * FROM cand_pt LIMIT 5;")

,candidate_nm,party_nm
0,"Rubio, Marco",Republican
1,"Santorum, Richard J.",Republican
2,"Perry, James R. (Rick)",Republican
3,"Carson, Benjamin S.",Republican
4,"Cruz, Rafael Edward 'Ted'",Republican


In [76]:
# left join 
exp_dtls_w_pt = mysql('''SELECT e.cand_nm
            , e.disb_amt
            , (case when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='Jan' 
                        then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-01-'|| substr(e.disb_dt,-2,2)
            when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='Feb' 
                then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-02-'|| substr(e.disb_dt,-2,2)
            when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='Mar' 
                then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-03-'|| substr(e.disb_dt,-2,2)
            when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='Apr' 
                then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-04-'|| substr(e.disb_dt,-2,2)
            when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='May' 
                then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-05-'|| substr(e.disb_dt,-2,2)
            when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='Jun' 
                then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-06-'|| substr(e.disb_dt,-2,2)
            when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='Jul' 
                then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-07-'|| substr(e.disb_dt,-2,2)
            when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='Aug' 
                then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-08-'|| substr(e.disb_dt,-2,2)
            when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='Sep' 
                then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-09-'|| substr(e.disb_dt,-2,2)
            when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='Oct' 
                then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-10-'|| substr(e.disb_dt,-2,2)
            when substr(e.disb_dt,instr(e.disb_dt,'-')+1,3) ='Nov' 
                then substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-11-'|| substr(e.disb_dt,-2,2)
            else substr(e.disb_dt,1,instr(e.disb_dt,'-')-1) || '-12-'|| substr(e.disb_dt,-2,2) end ) AS disb_dt
            , e.recipient_st
            , c.party_nm
        FROM exp_dtls AS e
        left join cand_pt AS c
            on e.cand_nm = c.candidate_nm;
        ''')

In [97]:
mysql('''
SELECT * 
FROM exp_dtls_w_pt
LIMIT 10;
''')

,cand_nm,disb_amt,disb_dt,recipient_st,party_nm
0,"Rubio, Marco",500.00,15-09-15,AK,Republican
1,"Rubio, Marco",2500.00,16-11-15,AK,Republican
2,"Rubio, Marco",1000.00,30-09-15,AK,Republican
3,"Rubio, Marco",336.09,18-12-15,AL,Republican
4,"Rubio, Marco",10000.00,22-10-15,AL,Republican
5,"Rubio, Marco",500.00,3-12-15,AL,Republican
6,"Rubio, Marco",250.00,23-11-15,AL,Republican
7,"Rubio, Marco",857.50,29-02-16,AL,Republican
8,"Rubio, Marco",381.50,18-03-16,AL,Republican
9,"Rubio, Marco",313.50,27-02-16,AL,Republican


In [86]:
mysql('''
SELECT cand_nm
    , disb_mo
    , disb_yr
    , monthly_total
    , RANK() OVER (
        PARTITION BY cand_nm
        ORDER BY monthly_total desc
    ) AS monthly_total_rank
    , SUM(monthly_total) OVER (
        PARTITION BY cand_nm
        ORDER BY disb_mo          
        ) as montly_rolling_total
    
FROM

(SELECT cand_nm
        , substr(disb_dt,-2,3) AS disb_yr
        , ltrim(substr(disb_dt,instr(disb_dt,'-')+1,2),'0')*1  AS disb_mo
        , SUM(disb_amt) AS monthly_total
    FROM exp_dtls_w_pt 
    GROUP BY 1, 2, 3
    order by 1, 2, 3)
    limit 20;''')


,cand_nm,disb_mo,disb_yr,monthly_total,monthly_total_rank,montly_rolling_total
0,"Bush, Jeb",1,16,6252843.24,1,6252843.24
1,"Bush, Jeb",7,15,4603528.79,2,17942757.81
2,"Bush, Jeb",8,15,4351502.26,3,22294260.07
3,"Bush, Jeb",10,15,4274334.51,4,29045386.34
4,"Bush, Jeb",11,15,3316765.65,5,32362151.99
5,"Bush, Jeb",2,16,3070376.46,6,9323219.70
6,"Bush, Jeb",6,15,3061192.61,7,13339229.02
7,"Bush, Jeb",9,15,2476791.76,8,24771051.83
8,"Bush, Jeb",12,15,2132243.14,9,34494395.13
9,"Bush, Jeb",3,16,954816.71,10,10278036.41


In [89]:
# using windows functions
exp_ttls_w_pt=mysql('''
WITH cand_amt as (
SELECT cand_nm
    , disb_mo
    , disb_yr
    , monthly_total
    , RANK() OVER (
        PARTITION BY cand_nm
        ORDER BY monthly_total desc
    ) AS monthly_total_rank
    , SUM(monthly_total) OVER (
        PARTITION BY cand_nm
        ORDER BY disb_mo          
        ) as montly_rolling_total
    
FROM

(SELECT cand_nm
        , substr(disb_dt,-2,3) AS disb_yr
        , ltrim(substr(disb_dt,instr(disb_dt,'-')+1,2),'0')*1  AS disb_mo
        , SUM(disb_amt) AS monthly_total
    FROM exp_dtls_w_pt 
    GROUP BY 1, 2, 3
    order by 1, 2, 3)
)
SELECT ca.cand_nm
    , ca.disb_mo
    , ca.disb_yr
    , ca.monthly_total
    , ca.monthly_total_rank
    , ca.montly_rolling_total
    , cp.party_nm as cand_party

FROM cand_amt as ca
LEFT JOIN cand_pt as cp
    ON ca.cand_nm = cp.candidate_nm
ORDER BY cand_nm, disb_mo
''')

In [90]:
mysql('select * from exp_ttls_w_pt limit 20')

,cand_nm,disb_mo,disb_yr,monthly_total,monthly_total_rank,montly_rolling_total,cand_party
0,"Bush, Jeb",1,16,6252843.24,1,6252843.24,Republican
1,"Bush, Jeb",2,16,3070376.46,6,9323219.70,Republican
2,"Bush, Jeb",3,16,954816.71,10,10278036.41,Republican
3,"Bush, Jeb",6,15,3061192.61,7,13339229.02,Republican
4,"Bush, Jeb",7,15,4603528.79,2,17942757.81,Republican
5,"Bush, Jeb",8,15,4351502.26,3,22294260.07,Republican
6,"Bush, Jeb",9,15,2476791.76,8,24771051.83,Republican
7,"Bush, Jeb",10,15,4274334.51,4,29045386.34,Republican
8,"Bush, Jeb",11,15,3316765.65,5,32362151.99,Republican
9,"Bush, Jeb",12,15,2132243.14,9,34494395.13,Republican
